In [ ]:
from supabase import create_client
import os

client = create_client(supabase_url=os.getenv('SUPABASE_URL'), supabase_key=os.getenv('SUPABASE_SERVICE_ROLE_KEY'))

In [ ]:
client.table('Building').insert({"name": "test", "gfa": 1000,}).execute()
client.table("DesignVector").insert({"name": "testdesign"}).execute()
client.table("DemandScenario").insert({"name": "testdemand"}).execute()

In [ ]:
client.table("DemandScenarioBuilding").insert({"demand_scenario_id": 1, "building_id": 1, "design_vector_id": 1}).execute()

In [ ]:
import numpy as np
client.table("BuildingSimulationResult").insert({
	"id": 1,
	"heating": np.random.rand(8760).tolist(),
	"cooling": np.random.rand(8760).tolist(),
	"lighting": np.random.rand(8760).tolist(),
	"equipment": np.random.rand(8760).tolist(),
	"pumps": np.random.rand(8760).tolist(),
	"fans": np.random.rand(8760).tolist(),
	"water": np.random.rand(8760).tolist(),
	"misc": np.random.rand(8760).tolist(),

}).execute()

In [ ]:
res = client.table("BuildingSimulationResult").select("*").execute()
import json
np.array(json.loads(res.data[0]["heating"])).shape

In [ ]:
from pydantic import BaseModel, Field, validate_call
from datetime import datetime

class BuildingBase(BaseModel, extra="forbid"):
   name: str = Field(..., description="Name of the building")
   gfa: float = Field(..., gt=100, description="Gross Floor Area of the building, m2")

class Building(BuildingBase):
   id: int = Field(..., description="Unique identifier for the building")
   created_at: str = Field(..., description="Timestamp of the creation of the building")

   @classmethod
   def get(cls, id: int):
      building = client.table("Building").select("*").eq("id", id).execute()
      if len(building.data) == 0:
         raise ValueError(f"Building with id {id} not found")

      return cls(**building.data[0])
   
   @classmethod
   @validate_call()
   def create(cls, building: BuildingBase):
      building = client.table("Building").insert(building.model_dump()).execute()
      return cls(**building.data[0])

   
   def commit(self):
      client.table("Building").upsert(self.model_dump()).execute()
   

building = Building.get(1)
building.gfa = 2000
building.commit()

building = Building.create({"name": "test2", "gfa": 3000})
building.id

In [ ]:
from pydantic import model_validator
from typing import Optional
import pandas as pd
class BuildingSimulationResult(BaseModel, arbitrary_types_allowed=True, extra="forbid"):
    id: int
    created_at: Optional[str] = Field(None, description="Timestamp of the creation of the record")
    heating: np.ndarray
    cooling: np.ndarray
    lighting: np.ndarray
    equipment: np.ndarray
    pumps: np.ndarray
    fans: np.ndarray
    water: np.ndarray
    misc: np.ndarray

    # set up all np.ndarrays to serialize to list
    class Config:
        json_encoders = {
            np.ndarray: lambda v: v.tolist()
        }
    
    


    @model_validator(mode="before")
    def cast_fields_to_numpy(cls, v):
        for key in v:
            if cls.model_fields[key].annotation == np.ndarray:
                v[key] = np.array(json.loads(v[key]))
                assert v[key].shape == (8760,), f"Field {key} must have shape (8760,) but has shape {v[key].shape}"
        return v

    @classmethod
    def get(cls, id: int):
        res = client.table("BuildingSimulationResult").select("*").eq("id", id).execute()
        if len(res.data) == 0:
            raise ValueError(f"BuildingSimulationResult with id {id} not found")

        return cls(**res.data[0])
    
    def to_df(self) -> pd.DataFrame:
        df = pd.DataFrame({
            "heating": self.heating,
            "cooling": self.cooling,
            "lighting": self.lighting,
            "equipment": self.equipment,
            "pumps": self.pumps,
            "fans": self.fans,
            "water": self.water,
            "misc": self.misc,
        }, index=pd.date_range(start="2024-01-01 00:00:00", periods=8760, freq="H", name="timestep") )
        df = df.set_index(pd.Series([self.id]*8760, name="building_id"), append=True, )
        df = df.unstack(level="timestep")
        return df
    
    def from_df(self, df: pd.DataFrame):
        series = df.loc[self.id]
        self.heating = series.heating.values
        self.cooling = series.cooling.values
        self.lighting = series.lighting.values
        self.equipment = series.equipment.values
        self.pumps = series.pumps.values
        self.fans = series.fans.values
        self.water = series.water.values
        self.misc = series.misc.values

    def commit(self):
        client.table("BuildingSimulationResult").upsert(self.model_dump(mode="json")).execute()

building_result = BuildingSimulationResult.get(1).to_df()
building_result["heating"] = 1
building_r = BuildingSimulationResult.get(1)
building_r.from_df(building_result)
building_r.commit()